In [1]:
import pandas as pd
import numpy as np
import json
import requests
import settings

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions

In [2]:
token = 'Bearer ' + settings.access_token
base_url = "http://api.genius.com"
headers = {'Authorization': token}

# Post

In [38]:
url = 'https://genius.com/albums/Billie-eilish/When-we-all-fall-asleep-where-do-we-go'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
track_list = soup.find_all('div', {'class' : 'chart_row-content'})
songs = []

for track in track_list:
    album = {}
    song = track.find('h3').text.strip().split('\n').pop(0)
    album['song']= song.replace('\xa0', '')
    songs.append(album)
    album['link'] = track.find('a').attrs['href']
    
    post = pd.DataFrame(songs)

album_lyrics = []
def get_url(url):
    song_lyrics = {}
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    [h.extract() for h in html('script')]
    lyrics = html.find('div', class_ = 'lyrics').get_text().strip().split("\n")
    
    #need to figure out how to remove bad chars

    song_lyrics['lyrics'] = lyrics
    album_lyrics.append(song_lyrics)
    
for links in post['link']:
    get_url(links)

In [39]:
post_album = pd.DataFrame(album_lyrics)
post2019 = pd.concat([post, post_album], axis = 1)
post2019.to_csv('data/post2019.csv', index = False)
post2019

,song,link,lyrics
0,!!!!!!!,https://genius.com/Billie-eilish--lyrics,"[[Spoken: Billie Eilish & FINNEAS], My Invisal..."
1,​bad guy,https://genius.com/Billie-eilish-bad-guy-lyrics,"[[Verse 1], White shirt now red, my bloody nos..."
2,​xanny,https://genius.com/Billie-eilish-xanny-lyrics,"[[Verse 1], What is it about them?, I must be ..."
3,​you should see me in a crown,https://genius.com/Billie-eilish-you-should-se...,"[[Verse 1], Bite my tongue, bide my time, Wear..."
4,​all the good girls go to hell,https://genius.com/Billie-eilish-all-the-good-...,"[[Intro], My Lucifer is lonely, , [Verse 1], S..."
5,​wish you were gay,https://genius.com/Billie-eilish-wish-you-were...,"[[Verse 1], ""Baby, I don't feel so good,"" six ..."
6,​when the party's over,https://genius.com/Billie-eilish-when-the-part...,"[[Verse 1], Don't you know I'm no good for you..."
7,8,https://genius.com/Billie-eilish-8-lyrics,"[[Verse 1], Wait a minute, let me finish, I kn..."
8,​my strange addiction,https://genius.com/Billie-eilish-my-strange-ad...,"[[Intro], No, Billy, I haven't done that dance..."
9,​bury a friend,https://genius.com/Billie-eilish-bury-a-friend...,"[[Intro: Mehki Raine], Billie, , [Chorus: Bill..."


# Billie

In [32]:
url = 'https://genius.com/albums/Billie-eilish/When-we-all-fall-asleep-where-do-we-go'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
track_list = soup.find_all('div', {'class' : 'chart_row-content'})
songs = []

for track in track_list:
    album = {}
    song = track.find('h3').text.strip().split('\n').pop(0)
    album['song']= song.replace('\xa0', '')
    songs.append(album)
    album['link'] = track.find('a').attrs['href']
    
    billie = pd.DataFrame(songs)

album_lyrics = []
def get_url(url):
    song_lyrics = {}
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    [h.extract() for h in html('script')]
    lyrics = html.find('div', class_ = 'lyrics').get_text().strip().split("\n")
    
    #need to figure out how to remove bad chars

    song_lyrics['lyrics'] = lyrics
    album_lyrics.append(song_lyrics)
    
for links in billie['link']:
    get_url(links)

In [34]:
billie_album = pd.DataFrame(album_lyrics)
billie2019 = pd.concat([billie, billie_album], axis = 1)
billie2019.to_csv('data/billie2019.csv', index = False)
billie2019

,song,link,lyrics
0,!!!!!!!,https://genius.com/Billie-eilish--lyrics,"[[Spoken: Billie Eilish & FINNEAS], My Invisal..."
1,​bad guy,https://genius.com/Billie-eilish-bad-guy-lyrics,"[[Verse 1], White shirt now red, my bloody nos..."
2,​xanny,https://genius.com/Billie-eilish-xanny-lyrics,"[[Verse 1], What is it about them?, I must be ..."
3,​you should see me in a crown,https://genius.com/Billie-eilish-you-should-se...,"[[Verse 1], Bite my tongue, bide my time, Wear..."
4,​all the good girls go to hell,https://genius.com/Billie-eilish-all-the-good-...,"[[Intro], My Lucifer is lonely, , [Verse 1], S..."
5,​wish you were gay,https://genius.com/Billie-eilish-wish-you-were...,"[[Verse 1], ""Baby, I don't feel so good,"" six ..."
6,​when the party's over,https://genius.com/Billie-eilish-when-the-part...,"[[Verse 1], Don't you know I'm no good for you..."
7,8,https://genius.com/Billie-eilish-8-lyrics,"[[Verse 1], Wait a minute, let me finish, I kn..."
8,​my strange addiction,https://genius.com/Billie-eilish-my-strange-ad...,"[[Intro], No, Billy, I haven't done that dance..."
9,​bury a friend,https://genius.com/Billie-eilish-bury-a-friend...,"[[Intro: Mehki Raine], Billie, , [Chorus: Bill..."


# Ariana

In [36]:
url = 'https://genius.com/albums/Ariana-grande/Thank-u-next'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
track_list = soup.find_all('div', {'class' : 'chart_row-content'})
songs = []

for track in track_list:
    album = {}
    song = track.find('h3').text.strip().split('\n').pop(0)
    album['song']= song.replace('\xa0', '')
    songs.append(album)
    album['link'] = track.find('a').attrs['href']
    
    ariana = pd.DataFrame(songs)

album_lyrics = []
def get_url(url):
    song_lyrics = {}
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    [h.extract() for h in html('script')]
    lyrics = html.find('div', class_ = 'lyrics').get_text().strip().split("\n")
    
    #need to figure out how to remove bad chars

    song_lyrics['lyrics'] = lyrics
    album_lyrics.append(song_lyrics)
    
for links in ariana['link']:
    get_url(links)

In [37]:
ariana_album = pd.DataFrame(album_lyrics)
ariana2019 = pd.concat([ariana, ariana_album], axis = 1)
ariana2019.to_csv('data/ariana2019.csv', index = False)
ariana2019

,song,link,lyrics
0,​imagine,https://genius.com/Ariana-grande-imagine-lyrics,"[[Verse 1], Step up, the two of us, nobody kno..."
1,​needy,https://genius.com/Ariana-grande-needy-lyrics,"[[Verse 1], If you take too long to hit me bac..."
2,NASA,https://genius.com/Ariana-grande-nasa-lyrics,"[[Intro: Shangela Laquifa Wadley], This is one..."
3,​bloodline,https://genius.com/Ariana-grande-bloodline-lyrics,"[[Intro: Marjorie Grande], Because I'm trying ..."
4,​fake smile,https://genius.com/Ariana-grande-fake-smile-ly...,"[[Intro: Wendy Rene], After laughter comes tea..."
5,​bad idea,https://genius.com/Ariana-grande-bad-idea-lyrics,"[[Verse 1], I've been outta sight, I've been w..."
6,​make up,https://genius.com/Ariana-grande-make-up-lyrics,"[[Verse 1], I like to fuck with you just to ma..."
7,​ghostin,https://genius.com/Ariana-grande-ghostin-lyrics,"[[Verse 1], I know you hear me when I cry, I t..."
8,​in my head,https://genius.com/Ariana-grande-in-my-head-ly...,"[[Intro: Doug Middlebrook], Here's the thing: ..."
9,7 rings,https://genius.com/Ariana-grande-7-rings-lyrics,"[[Verse 1], Yeah, breakfast at Tiffany's and b..."
